# IndeedScrap doc
These functions allow to perform web scrapping on Indeed platform, to collect job detail.
____
Requirements:

In [1]:
from urllib.request import urlopen, HTTPError
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re

The *scrapPage()* function allows to scrap an html document from the URL.  
This is to avoid redundant code in the main function.

In [2]:
def scrapPage(url):
    with urlopen(url) as response:
        page = BeautifulSoup(response.read(), 'html.parser')
    return page

The *scrapID()* function collects the IDs of the job ads published on the active page.  
This information is the argument for the *'data-jk'* attribute in *'jobsearch-SerpJobCard'* class divisions

In [3]:
def scrapID(page):
    resultCol = page.find(id="resultsCol")
    setID = {
        jobcard["data-jk"]
        for jobcard in resultCol.findAll("div",
                                         {"class": "jobsearch-SerpJobCard"})
    }
    return setID

The *stripmatch()* function gets the match and the number of pages visited for the current search.  
Since the form of a number in thousands differs from one country to another, regular expressions are used to harmonize the result: a list greater than two means a result greater than one thousand.

In [4]:
def stripmatch(page):
    try:
        text = page.find(id="searchCountPages").text.strip()
    except AttributeError:
        repage = match = None
    else:
        numlist = [num for num in re.findall(r'-?\d+\.?\d*', text)]
        repage = int(numlist[0])
        if len(numlist) == 2:
            match = int(numlist[1])
        else:
            match = int(''.join(numlist[1:]))
    return repage, match

The **`scrapIndeedID()`** function extracts the IDs for each job for each country searched for.  
The site is divided into different country-independent subdomains (the site in one country does not have access to the data in the other), scraping is performed for each subdomain of the site.  
The number of results per page is arbitrarily set to 50.  
After page 101 of results, Indeed considers the ads to be irrelevant. These will not be kept.

In [5]:
def scrapIndeedID(searchList, countryList):
    setID = set()
    for search in searchList:
        search = search.replace(" ", "+")
        for country in countryList:
            country = country.lower()
            listID = set()
            limit = 50
            start = repage = count = 0
            match = None
            while (repage <= 101 or len(listID) < match):
                url = "https://{}.indeed.com/jobs?q={}&limit={}&start={}".format(
                    country, search, limit, start)
                try:
                    page = scrapPage(url)
                except HTTPError:
                    break
                else:
                    repage, match = stripmatch(page)
                    count += 1
                    if (match is None or repage < count):
                        break
                    else:
                        listID = listID.union({(country, jobID)
                                               for jobID in list(scrapID(page))
                                               })
                        start += limit
            setID = setID.union(listID)
    return setID

The *dicoFromScrap()* function extracts the desired data from a tuple of the country and the *jobID*. A scraping is then performed for each page of a job. The collected information is:

- description: long description
- country: 2-letter abbreviation of the country in the tuple
- city: full city name
- posted: job post creation date, precise date for job published in the last 30 days
- header: post title
- company: company name
- type\*: type of employment contract (employee, intern...)
- category\*: job category
- url: post url redirection


\* As the information is not formatted by most sub-domains, it will be extracted using word processing algorithms.

In [6]:
def dicoFromScrap(tupleID):
    dico = {}
    url = "https://www.indeed.com/viewjob?jk={}".format(tupleID[1])
    try:
        page = scrapPage(url)
    except HTTPError:
        return dico

    def postedDate(page):
        date = int(
            re.findall(
                r'-?\d+\.?\d*',
                page.find("div", {
                    "class": "jobsearch-JobMetadataFooter"
                }).text)[0])
        posted = (datetime.now() +
                  timedelta(days=-date)).isoformat(timespec='seconds')
        if date == 30: posted = "+ " + posted
        return posted

    dico["country"] = tupleID[0].upper()
    dico["url"] = url
    dico["description"] = page.find(id="jobDescriptionText").text
    dico["header"], dico["city"], *_ = page.head.title.text.split(" - ")
    dico["company"] = page.find("div", {"class": "icl-u-lg-mr--sm"}).text
    dico["type"] = dico["category"] = ""
    dico["posted"] = postedDate(page)

    return dico

In [7]:
dicoFromScrap(('fr', 'e84c3a7fa2589b2a'))

{'country': 'FR',
 'url': 'https://www.indeed.com/viewjob?jk=e84c3a7fa2589b2a',
 'description': 'Vous êtes rattaché(e) à notre Business Analyst – Revenue Manager, et intégré(e) à l’équipe Marketing.\nVos responsabilités\nRépondre aux besoins des équipes métiers\nVous participerez aux réunions préparatoires de création de nouveaux outils pour les équipes métiers\nVous assisterez les équipes métiers dans la définition des indicateurs de performance (KPI) pertinents\nCréation et amélioration des outils et rapports\nVous travaillerez en étroite collaboration avec notre équipe Dévs pour utiliser notre base de données\nVous serez force de proposition pour créer des tableaux de bord et études pour les différentes équipes métiers\n\nDurant votre stage, vous êtes encouragé(e) à vous investir sur les projets qui vous inspirent et à travailler en collaboration avec les autres équipes, pour créer des outils ou des nouvelles pratiques visant à l’amélioration du produit Zenpark.\nProfil recherché\nD